## Import libraries

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import os

# Import new customer data

In [2]:
path = r'/Users/rusamijan/03-2023 Instacart Basket Analysis'

In [3]:
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

# Task 4.9 (Part 1)

## 2. Wrangle the data so that it follows consistent logic; for example, rename columns with illogical names and drop columns that don’t add anything to your analysis.

In [ ]:
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [ ]:
df_cust.shape

(206209, 8)

### Rename columns

In [5]:
df_cust.rename(columns = {'First Name': 'first_name','Surnam': 'surname','Gender': 'gender', 'STATE': 'state', 'Age': 'age', 'date_joined' :'date_of_registration', 'n_dependants' : 'number_dependants', 'fam_status' : 'marital_status'}, inplace = True)

### Dropping columns that don't add anything to analysis

In [6]:
df_cust = df_cust.drop(columns = ['first_name','surname'])

In [7]:
df_cust.head()

,user_id,gender,state,age,date_of_registration,number_dependants,marital_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [25]:
df_cust.shape

(206209, 8)

## 3. Complete the fundamental data quality and consistency checks 

### Fnding null value in product

In [8]:
df_cust.isnull().sum()

user_id                 0
gender                  0
state                   0
age                     0
date_of_registration    0
number_dependants       0
marital_status          0
income                  0
dtype: int64

#### no missing values in the remaining columns of the dataset

### Find duplicate values and keep in df_dups

In [9]:
df_dups = df_cust[df_cust.duplicated()]

In [10]:
df_dups

,user_id,gender,state,age,date_of_registration,number_dependants,marital_status,income


#### no duplicate data

### Check order for mixed type

In [11]:
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col, ' mixed')
  else: print(col, ' consistent')

user_id  consistent
gender  consistent
state  consistent
age  consistent
date_of_registration  consistent
number_dependants  consistent
marital_status  consistent
income  consistent


#### no mixed type data

## 4. Combine your customer data with the rest of your prepared Instacart data. (Hint: Make sure the key columns are the same data type!)

### Import recent data -> "df_orders_products_merged_updated_2"

In [12]:
df_ords_prods_merged_new = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_orders_products_merged_updated_2.pkl'))

In [13]:
df_ords_prods_merged_new.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_prior_orders,order_frequency_flag
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,Mid-range product,Regularly busy,Regular Days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,...,Mid-range product,Regularly busy,Slowest Days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,...,Mid-range product,Regularly busy,Slowest Days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,...,Mid-range product,Least busy,Slowest Days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,...,Mid-range product,Least busy,Slowest Days,Most Orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [14]:
df_ords_prods_merged_new.shape

(32404859, 24)

In [15]:
df_cust.shape

(206209, 8)

### Check data type

In [16]:
df_cust['user_id'].dtype

dtype('int64')

In [17]:
df_ords_prods_merged_new['user_id'].dtype

dtype('int64')

#### Both data types are the same and ready to combine

### Combining customer data and df_ords_prods_merged_new by using user_id and outer join

In [19]:
df_ords_prods_custm = pd.merge(df_ords_prods_merged_new, df_cust, on = 'user_id', how = 'outer')

In [20]:
df_ords_prods_custm.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_prior_orders,order_frequency_flag,gender,state,age,date_of_registration,number_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196.0,1.0,1.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196.0,1.0,1.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196.0,1.0,1.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196.0,1.0,1.0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


In [21]:
df_ords_prods_custm['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [22]:
df_ords_prods_custm.shape

(32404859, 31)

### Export update data

In [23]:
df_ords_prods_custm.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers_all.pkl'))